In [1]:
import streamlit as st
import csv
import pandas as pd
import plotly.express as px
import folium
from folium import plugins

## BICICLETAS

In [2]:
bicis = pd.read_csv('aparcaments-bicicletes-aparcamientos-bicicletas.csv', sep = ';')
bicis = bicis.drop(columns = ['OBJECTID','created_user','created_date','last_edited_user','last_edited_date','geo_shape'])
bicis[['lat','lon']] =bicis.geo_point_2d.str.split(pat=',', expand = True)
bicis = bicis.drop(columns = ['geo_point_2d'])
bicis['lat'] = bicis['lat'].astype(float)
bicis['lon'] = bicis['lon'].astype(float)
bicis

,Tipus / Tipo,Nombre Places / Número Plazas,lat,lon
0,A - Horquilla,6,39.459078,-0.385449
1,A - Horquilla,8,39.471800,-0.324463
2,A - Horquilla,4,39.462329,-0.369685
3,A - Horquilla,8,39.476439,-0.364275
4,A - Horquilla,10,39.494033,-0.366231
...,...,...,...,...
3445,A - Horquilla,6,39.461654,-0.371580
3446,A - Horquilla,10,39.487174,-0.368874
3447,A - Horquilla,6,39.464633,-0.409697
3448,A - Horquilla,10,39.458098,-0.351317


In [5]:
mapa = folium.Map(location=[39.4697500, -0.3773900], zoom_start=12)
cluster = plugins.MarkerCluster().add_to(mapa)

for index, row in bicis.iterrows():
    lat = row['lat']
    lon = row['lon']
    text = 'Numero de plazas: '+str(row['Nombre Places / Número Plazas'])
    folium.Marker(location=[lat,lon], popup=text, icon = folium.Icon(color='orange',prefix = 'fa', icon = 'bicycle')).add_to(cluster)
plugins.Geocoder().add_to(mapa)

mapa.save('mapa-bicicletas.html')


## MOTOS

In [7]:
motos = pd.read_csv('aparcament-per-a-motos-aparcamiento-para-motos.csv', sep = ';')
motos = motos.drop(columns = ['Carrer / Calle','OBJECTID','created_user','created_date','last_edited_user','last_edited_date','geo_shape'])
motos[['lat','lon']] =motos.geo_point_2d.str.split(pat=',', expand = True)
motos = motos.drop(columns = ['geo_point_2d'])
motos['lat'] = motos['lat'].astype(float)
motos['lon'] = motos['lon'].astype(float)
mapa = folium.Map(location=[39.4697500, -0.3773900], zoom_start=12)
cluster = plugins.MarkerCluster().add_to(mapa)

for index, row in motos.iterrows():
    lat = row['lat']
    lon = row['lon']
    text = 'Numero de plazas: '+str(int(row['Nombre Places / Número Plazas']))
    folium.Marker(location=[lat,lon], popup=text, icon = folium.Icon(color='green',prefix = 'fa', icon = 'motorcycle')).add_to(cluster)
plugins.Geocoder().add_to(mapa)

mapa.save('mapa-motos.html')

## COCHES

In [8]:
ora = pd.read_csv("aparcaments-ora-aparcamientos-ora.csv",sep = ";")
mov = pd.read_csv("aparcaments-persones-mobilitat-reduida-aparcamientos-personas-movilidad-reducida.csv",sep=";")
vados = pd.read_csv("guals-vados.csv",sep=";")
parkings = pd.read_csv("parkings.csv",sep=";")
carg = pd.read_csv("zones-carrega-descarrega-zonas-carga-descarga.csv",sep=";")

In [9]:
def normalize_df(df):
    df["lat"] = [float(df["geo_point_2d"][i].split(",")[0]) for i in range(len(df["geo_point_2d"]))]
    df["lon"] = [float(df["geo_point_2d"][i].split(",")[1]) for i in range(len(df["geo_point_2d"]))]

    return df

In [10]:
ora = normalize_df(ora)
mov = normalize_df(mov)
vados = normalize_df(vados)
carg = normalize_df(carg)
parkings = normalize_df(parkings)

In [11]:
m = folium.Map(location=[39.469, -0.377],zoom_start=13)
plugins.Geocoder().add_to(m)

cluster1 = plugins.MarkerCluster().add_to(folium.FeatureGroup(name = 'Vados').add_to(m))
for lat,lon in zip(vados["lat"],vados["lon"]):
    folium.Marker(location = [lat,lon], icon = folium.Icon(color='lightred',prefix = "fa",icon = 'times')).add_to(cluster1)

cluster2 = plugins.MarkerCluster().add_to(folium.FeatureGroup(name = 'Parkings').add_to(m))
for lat,lon in zip(parkings["lat"],parkings["lon"]):
    folium.Marker(location = [lat,lon], icon = folium.Icon(color='lightblue',prefix = "fa",icon = 'product-hunt')).add_to(cluster2)

cluster3 = plugins.MarkerCluster().add_to(folium.FeatureGroup(name = 'Zonas de Carga/Descarga').add_to(m))
for lat,lon in zip(carg["lat"],carg["lon"]):
    folium.Marker(location = [lat,lon], icon = folium.Icon(color='darkgreen',prefix = "fa",icon = 'truck')).add_to(cluster3)

cluster4 = plugins.MarkerCluster().add_to(folium.FeatureGroup(name = 'Zonas personas con discapacidad').add_to(m))
for lat,lon in zip(mov["lat"],mov["lon"]):
    folium.Marker(location = [lat,lon], icon = folium.Icon(color='pink',prefix = "fa",icon = 'wheelchair')).add_to(cluster4)

cluster5 = plugins.MarkerCluster().add_to(folium.FeatureGroup(name = 'Zonas con limitación horaria').add_to(m))
for color,lat,lon in zip(ora["Color"],ora["lat"],ora["lon"]):
  if color == "Azul":
    folium.Marker(location = [lat,lon], tooltip = folium.Tooltip(text = "Zona azul"), icon = folium.Icon(color='blue',prefix = "fa",icon = 'clock-o')).add_to(cluster5)
  elif color == "Amarillo":
    folium.Marker(location = [lat,lon], tooltip = folium.Tooltip(text = "Zona amarilla"), icon = folium.Icon(color='beige',prefix = "fa",icon = 'clock-o')).add_to(cluster5)
  elif color == "Blanco":
    pass
  elif color == "Naranja":
    folium.Marker(location = [lat,lon], tooltip = folium.Tooltip(text = "Zona naranja"), icon = folium.Icon(color='orange',prefix = "fa",icon = 'clock-o')).add_to(cluster5)
  else:
    folium.Marker(location = [lat,lon], tooltip = folium.Tooltip(text = "Zona verde"), icon = folium.Icon(color='green',prefix = "fa",icon = 'clock-o')).add_to(cluster5)


folium.LayerControl().add_to(m);

m.save('mapa-coches.html')